# Trading bot 
made by Rafael Roane and Raphael van den Doel
using binance

Starting amount: 114 USD

v0.0

### Config

In [2]:
api_key = 'xxxxxxxxxxx'
api_secret = 'xxxxxxxxx'
pair = 'ETHBUSD'

### Start of strategy

In [3]:
import sqlalchemy
import pandas as pd
from binance.client import Client

In [5]:
client = Client(api_key,api_secret)

In [6]:
engine = sqlalchemy.create_engine('sqlite:///stream.db')

In [33]:
def walletBalance():
    dfWallet = pd.DataFrame(client.get_account()["balances"])
    dfWallet["free"] = dfWallet["free"].astype(float).round(4)
    df = dfWallet[dfWallet["free"] > 0]
    print(df)

    asset      free      locked
188  BUSD  113.8529  0.00000000


In [ ]:
def strategy(entry, lookback, qty, open_position=False):
    while True:
        df = pd.read_sql(pair, engine)
        lookbackperiod = df.iloc[-lookback:]
        cumret = (lookbackperiod.Price.pct_change() +1).cumprod() - 1
        if not open_position:
            if cumret[cumret.last_valid_index()] > entry:
                print("Buying...")
                print(df.iloc[-1:])
                order = client.create_order(symbol=pair,
                                           side='BUY',
                                           type='MARKET',
                                           quantity=qty)
                print(order)
                open_position = True
                break
    if open_position:
        while True:
            df = pd.read_sql(pair, engine)
            sincebuy = df.loc[df.Time > 
                              pd.to_datetime(order['transactTime'],
                                            unit='ms')]
            if len(sincebuy) > 1:
                sincebuyret = (sincebuy.Price.pct_change() +1).cumprod() - 1
                last_entry = sincebuyret[sincebuyret.last_valid_index()]
                if last_entry > 0.0015 or last_entry < -0.0012:
                    order = client.create_order(symbol=pair,
                                           side='SELL',
                                           type='MARKET',
                                           quantity=qty)
                    print(order)
                    print("Selling...")
                    print(df.iloc[-1:])
                    walletBalance():
                    
                    break

    asset      free      locked
188  BUSD  113.8529  0.00000000


In [34]:
strategy(0.001, 30, 0.0001)

ordering ETH
683    3192.26
Name: Price, dtype: float64
683   2022-02-10 18:19:51.122
Name: Time, dtype: datetime64[ns]


ValueError: Can only compare identically-labeled Series objects